In [ ]:
#Training the dataset

#IMPORTING

import numpy as np
import tensorflow as tf
from time import time
import math

from include.data import get_data_set
from include.model import model, lr

#DECLARATION

train_x, train_y = get_data_set("train")
test_x, test_y = get_data_set("test")
tf.set_random_seed(21)
x, y, output, y_pred_cls, global_step, learning_rate = model()
global_accuracy = 0
epoch_start = 0


# PARAMS
_BATCH_SIZE = 128
_EPOCH = 60
_SAVE_PATH = "./tensorboard/cifar-10-v1.0.0/"
 
 
# LOSS AND OPTIMIZER
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-08).minimize(loss, global_step=global_step)
                                                             
                               
# PREDICTION AND ACCURACY CALCULATION
correct_prediction = tf.equal(y_pred_cls, tf.argmax(y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
 
# SAVER
merged = tf.summary.merge_all()
saver = tf.train.Saver()
sess = tf.Session()
train_writer = tf.summary.FileWriter(_SAVE_PATH, sess.graph)
 
 
try:
    print("Restoring the latest checkpoint ...")
    last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=_SAVE_PATH)
    saver.restore(sess, save_path=last_chk_path)
    print("Checkpoint Restored:", last_chk_path)
except ValueError:
    print("Failed to restore checkpoint. Initializing variables instead.")
    sess.run(tf.global_variables_initializer())
 
 
def train(epoch):
    global epoch_start
    epoch_start = time()
    batch_size = int(math.ceil(len(train_x) / _BATCH_SIZE))
    i_global = 0
 
    for s in range(batch_size):
        batch_xs = train_x[s*_BATCH_SIZE: (s+1)*_BATCH_SIZE]
        batch_ys = train_y[s*_BATCH_SIZE: (s+1)*_BATCH_SIZE]
 
        start_time = time()
        i_global, _, batch_loss, batch_acc = sess.run(
            [global_step, optimizer, loss, accuracy],
            feed_dict={x: batch_xs, y: batch_ys, learning_rate: lr(epoch)})
        duration = time() - start_time
 
        if s % 10 == 0:
            percentage = int(round((s/batch_size)*100))
 
            bar_len = 29
            filled_len = int((bar_len*int(percentage))/100)
            bar = '=' * filled_len + '>' + '-' * (bar_len - filled_len)
 
            msg = "Global step: {:>5} - [{}] {:>3}% - acc: {:.4f} - loss: {:.4f} - {:.1f} sample/sec"
            print(msg.format(i_global, bar, percentage, batch_acc, batch_loss, _BATCH_SIZE / duration))
 
    test_and_save(i_global, epoch)
 
 
 def test_and_save(_global_step, epoch):
    global global_accuracy
    global epoch_start
 
    i = 0
    predicted_class = np.zeros(shape=len(test_x), dtype=np.int)
    while i < len(test_x):
    j = min(i + _BATCH_SIZE, len(test_x)) 
    batch_xs = test_x[i:j, :]
    batch_ys = test_y[i:j, :] 
    predicted_class[i:j] = sess.run( y_pred_cls, feed_dict={x: batch_xs, y: batch_ys, learning_rate: lr(epoch)} ) 
    i = j 
    correct = (np.argmax(test_y, axis=1) == predicted_class)
    acc = correct.mean()*100 
    correct_numbers = correct.sum()
    hours, rem = divmod(time() - epoch_start, 3600) minutes,
    seconds = divmod(rem, 60)
    mes = " Epoch {} - accuracy: {:.2f}% ({}/{}) - time: {:0>2}:{:0>2}:{:05.2f}"
    print(mes.format((epoch+1), acc, correct_numbers, len(test_x), int(hours), int(minutes), seconds))
 
    if global_accuracy != 0 and global_accuracy < acc: 
        summary = tf.Summary(value=[ tf.Summary.Value(tag="Accuracy/test", simple_value=acc), ])
        train_writer.add_summary(summary, _global_step) 
        saver.save(sess, save_path=_SAVE_PATH, global_step=_global_step) 
        mes = "This epoch receive better accuracy: {:.2f} > {:.2f}. Saving session..."
        print(mes.format(acc, global_accuracy))
        global_accuracy = acc
 
    elif global_accuracy == 0:
        global_accuracy = acc
 
    print("-------------------------------------------")
 
 
def main():
    train_start = time()
 
    for i in range(_EPOCH):
        print("Epoch: {}/{}".format((i+1), _EPOCH))
        train(i)
 
    hours, rem = divmod(time() - train_start, 3600)
    minutes, seconds = divmod(rem, 60)
    mes = "Best accuracy pre session: {:.2f}, time: {:0>2}:{:0>2}:{:05.2f}"
    print(mes.format(global_accuracy, int(hours), int(minutes), seconds))
 
 
if __name__ == "__main__":
    main()
 
 
sess.close()

Epoch: 60/60


Global step: 23070 - [>--------------]   0% - acc: 0.9531 - 
loss: 1.5081 - 7045.4 sample/sec

Global step: 23080 - [>-----------]   2% - acc: 0.9453 - loss: 1.5159 - 7147.6 sample/sec

Global step: 23090 - [=>---------------]   7% - acc: 0.9844 - loss: 1.4764 - 7152.6 sample/sec

Global step: 23100 - [==>----------------]   9% - acc: 0.9297 - loss: 1.5307 - 7104.4 sample/sec

Global step: 23110 - [==>-------------]  17% - acc: 0.9141 - loss: 1.5462 - 7091.4 sample/sec

Global step: 23120 - [===>-------------]  15% - acc: 0.9297 - loss: 1.5314 - 7152.9 sample/sec

Global step: 23130 - [====>-------------]  20% - acc: 0.9297 - loss: 1.5307 - 7134.8 sample/sec

Global step: 23140 - [=====>--------------]  25% - acc: 0.9375 - loss: 1.5231 - 7140.0 sample/sec

Global step: 23150 - [=====>--------------]  30% - acc: 0.9297 - loss: 1.5301 - 7162.8 sample/sec

Global step: 23160 - [======>-----------]  32% - acc: 0.9531 - loss: 1.5080 - 7112.3 sample/sec

Global step: 23170 - [=======>----------]  37% - acc: 0.9609 - loss: 1.5000 - 7154.0 sample/sec

Global step: 23180 - [========>------------]  40% - acc: 0.9531 - loss: 1.5074 - 6862.2 sample/sec

Global step: 23190 - [========>------------]  42% - acc: 0.9609 - loss: 1.4993 - 7134.5 sample/sec

Global step: 23200 - [=========>----------]  45% - acc: 0.9609 - loss: 1.4995 - 7466.0 sample/sec

Global step: 23210 - [==========>-----------]  48% - acc: 0.9375 - loss: 1.5231 - 7116.7 sample/sec

Global step: 23220 - [===========>------------]  50% - acc: 0.9453 - loss: 1.5153 - 7124.1 sample/sec

Global step: 23230 - [===========>-----------]  52% - acc: 0.9375 - loss: 1.5233 - 7094.5 sample/sec

Global step: 23240 - [============>-----------]  53% - acc: 0.9219 - loss: 1.5387 - 7173.9 sample/sec

Global step: 23250 - [=============>---------]  56% - acc: 0.8828 - loss: 1.5769 - 7184.1 sample/sec

Global step: 23260 - [==============>--------]  59% - acc: 0.9219 - loss: 1.5383 - 7079.7 sample/sec

Global step: 23270 - [==============>----------]  61% - acc: 0.8984 - loss: 1.5618 - 6628.6 sample/sec

Global step: 23280 - [===============>---------]  64% - acc: 0.9453 - loss: 1.5151 - 7045.7 sample/sec

Global step: 23290 - [================>-------]  66% - acc: 0.9609 - loss: 1.4996 - 7189.0 sample/sec

Global step: 23300 - [=================>--------]  74% - acc: 0.9609 - loss: 1.4997 - 7065.4 sample/sec

Global step: 23310 - [=================>--------]  78% - acc: 0.8750 - loss: 1.5842 - 7112.8 sample/sec

Global step: 23320 - [==================>--------]  81% - acc: 0.9141 - loss: 1.5463 - 7157.2 sample/sec

Global step: 23330 - [===================>----------]  83% - acc: 0.9062 - loss: 1.5549 - 7159.3 sample/sec

Global step: 23340 - [====================>-------]  85% - acc: 0.9219 - loss: 1.5389 - 7124.4 sample/sec

Global step: 23350 - [====================>--------]  87% - acc: 0.9609 - loss: 1.5002 - 7175.4 sample/sec

Global step: 23360 - [=====================>-------]  89% - acc: 0.9766 - loss: 1.4842 - 7194.2 sample/sec

Global step: 23370 - [======================>-----]  90% - acc: 0.9375 - loss: 1.5231 - 7178.5 sample/sec

Global step: 23380 - [======================>-----]  91% - acc: 0.8906 - loss: 1.5695 - 7173.2 sample/sec

Global step: 23390 - [=======================>------]  92% - acc: 0.9375 - loss: 1.5225 - 7139.1 sample/sec

Global step: 23400 - [========================>-----]  93% - acc: 0.9844 - loss: 1.4768 - 7140.1 sample/sec

Global step: 23410 - [=========================>----]  94% - acc: 0.9766 - loss: 1.4840 - 7177.0 sample/sec

Global step: 23420 - [==========================>---]  95% - acc: 0.9062 - loss: 1.5542 - 7192.1 sample/sec

Global step: 23430 - [==========================>---]  97% - acc: 0.9297 - loss: 1.5313 - 7105.3 sample/sec

Global step: 23440 - [===========================>--]  98% - acc: 0.9297 - loss: 1.5301 - 7153.3 sample/sec

Global step: 23450 - [============================>-]  99% - acc: 0.9375 - loss: 1.5231 - 7175.7 sample/sec

Global step: 23460 - [=============================>] 100% - acc: 0.9250 - loss: 1.5362 - 10247.5 sample/sec


Epoch 60 - accuracy: 69.52% (6952/10000)

Session is saved

